In [1]:
import os
import os.path as osp
import mmcv
import mmdet
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
os.chdir('/home/damon/Github/mmlab/mmdetection/')
%load_ext autoreload
%autoreload 2
sys.path.append('/home/damon/detorch/')
from visualization_utils import *
from debug_utils import *
from multiprocessing import Pool

swap = lambda b: np.stack([b[:, 1], b[:, 0], b[:, 3], b[:, 2]], 1)
def draw_bb(img, boxes):
    # xywh
    b = np.copy(boxes)
    return draw_bounding_boxes_on_image_array(img, boxes=swap(b))

In [2]:
volatile = '/home/damon/volatile/VisDrone2019-DET/'
# volatile = '/home/damon/DATASETS/Drone2019/VisDrone2019-DET/'
train_mode = 'VisDrone2018-DET-train-patch'
val_mode = 'VisDrone2018-DET-val'
test_mode = 'VisDrone2018-DET-test-challenge'
volatile_train = osp.join(volatile, train_mode)

train_img = osp.join(volatile_train, 'images')
train_txt = osp.join(volatile_train, 'annotations')

# cocogt

In [55]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json

In [96]:
cocoGt = COCO('/home/damon/volatile/VisDrone2019-DET/VisDrone2018-DET-val-patch/annotations_val.json')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [97]:
j = '/home/damon/volatile/VisDrone2019-DET/VisDrone2018-DET-val-patch/annotations_val.json'
with open(j, 'r') as f:
    j = json.load(f)

In [99]:
j_images = j['images']
j_anno = j['annotations']

In [ ]:
ind = np.random.randint(0, len(j_images))
anno = j_anno[ind]
imginfo = [v for v in j_images if v['id'] == anno['image_id']][0]

img = osp.join(volatile_train, imginfo['file_name'])
bbox = anno['bbox']
bbox = np.asarray(bbox, np.int32)[None, :]
bbox[:, 2:] += bbox[:, :2]
ret = draw_bb(mmcv.imread(img), bbox)
visualize_image_array(ret)

In [169]:
ind = np.random.randint(0, 47)
img = j_images[ind]

annoinfo = [v for v in j_anno if v['image_id'] == img['id']]
box = [v['bbox'] for v in annoinfo]
box = np.asarray(box, np.int32)
box[:, 2:] += box[:, :2]

# read pkl

In [9]:
import pickle, json
import mmcv
from mmdet.datasets import get_dataset
from visdrone.utils import patch_utils

pklfile = 'work_dirs/valdir/result.pkl'
outjson = pklfile + '.json'

with open(pklfile, 'rb') as f:
    p = pickle.load(f)
with open(outjson, 'rb') as f:
    j = json.load(f)

cfg = 'work_dirs/valdir/ssd512res_atrous.py'
cfg = mmcv.Config.fromfile(cfg)
ds = get_dataset(cfg.data.test)
out = patch_utils.merge_patch(ds, p)

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
load_annotations time: 0.1s from /home/damon/volatile/VisDrone2019-DET/VisDrone2018-DET-val-patch/annotations_val.json


In [5]:
pklfile = 'work_dirs/valdir/result.pkl'
outjson = pklfile + '.json'

In [6]:
with open(pklfile, 'rb') as f:
    p = pickle.load(f)
with open(outjson, 'rb') as f:
    j = json.load(f)

In [8]:
j

[{'image_id': 13519579,
  'bbox': [386.2891845703125,
   673.7333374023438,
   24.362091064453125,
   46.86492919921875],
  'score': 0.9477258920669556,
  'category_id': 1},
 {'image_id': 13519579,
  'bbox': [343.780029296875,
   643.6727905273438,
   22.25244140625,
   42.96087646484375],
  'score': 0.9351436495780945,
  'category_id': 1},
 {'image_id': 13519579,
  'bbox': [765.1652221679688,
   642.60302734375,
   29.9451904296875,
   48.0767822265625],
  'score': 0.9273627400398254,
  'category_id': 1},
 {'image_id': 13519579,
  'bbox': [575.3214111328125,
   446.3582458496094,
   18.871337890625,
   38.013336181640625],
  'score': 0.9070930480957031,
  'category_id': 1},
 {'image_id': 13519579,
  'bbox': [373.7029724121094,
   585.8646240234375,
   21.550384521484375,
   44.1865234375],
  'score': 0.9029316306114197,
  'category_id': 1},
 {'image_id': 13519579,
  'bbox': [183.7845001220703,
   769.1105346679688,
   45.61137390136719,
   64.80645751953125],
  'score': 0.864990770816

In [9]:
cfg = 'work_dirs/valdir/ssd512res_atrous.py'
cfg = mmcv.Config.fromfile(cfg)
ds = get_dataset(cfg.data.test)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load_annotations time: 0.0s from /home/damon/volatile/VisDrone2019-DET/VisDrone2018-DET-val-patch/annotations_val.json


In [55]:
vidout = '/tmp/vidout/uav0000086_00000_v.txt'

In [60]:
ann_file = vidout
with open(ann_file, 'r') as f:
    labels = f.readlines()
    labels = [v.strip('\n') for v in labels]
    labels = [v.split(',') for v in labels]
    labels1 = np.asarray(labels)[:, :10].astype(np.float32)
    
gt = '/home/damon/volatile/VisDrone2019-VID/VisDrone2018-VID-val/annotations/uav0000086_00000_v.txt'
with open(gt, 'r') as f:
    labels = f.readlines()
    labels = [v.strip('\n') for v in labels]
    labels = [v.split(',') for v in labels]
    labels2 = np.asarray(labels)[:, :10].astype(np.float32)

In [61]:
labels1

array([[111.,  -1., 107., ...,   1.,  -1.,  -1.],
       [111.,  -1., 558., ...,   1.,  -1.,  -1.],
       [111.,  -1., 450., ...,   1.,  -1.,  -1.],
       ...,
       [ 83.,  -1.,  83., ...,   2.,  -1.,  -1.],
       [ 83.,  -1.,  58., ...,   2.,  -1.,  -1.],
       [ 83.,  -1., 215., ...,   2.,  -1.,  -1.]], dtype=float32)

In [82]:
np.where(labels1[:, 0] == 150)[0]

array([5389, 5390, 5391, 5392, 5393, 5394, 5395, 5396, 5397, 5398, 5399,
       5400, 5401, 5402, 5403, 5404, 5405, 5406, 5407, 5408, 5409, 5410,
       5411, 5412, 5413, 5414, 5415, 5416, 5417, 5418, 5419, 5420, 5421,
       5422, 5423, 5424])

In [83]:
labels1

array([[111.,  -1., 107., ...,   1.,  -1.,  -1.],
       [111.,  -1., 558., ...,   1.,  -1.,  -1.],
       [111.,  -1., 450., ...,   1.,  -1.,  -1.],
       ...,
       [ 83.,  -1.,  83., ...,   2.,  -1.,  -1.],
       [ 83.,  -1.,  58., ...,   2.,  -1.,  -1.],
       [ 83.,  -1., 215., ...,   2.,  -1.,  -1.]], dtype=float32)